In [1]:
import numpy as np
import pandas as pd
import gensim
import re
import nltk
import json
import sys
import datetime
import operator
import matplotlib.pyplot as plt
import math
import csv
import timeit
import time
import textacy
import en_core_web_md

from annoy import AnnoyIndex
from collections import Counter
from nltk.corpus import stopwords
from nltk import word_tokenize, ngrams
from gensim import corpora, models, similarities
from gensim.models.doc2vec import Doc2Vec
from gensim.models import doc2vec
from pylab import plot, show, subplot, specgram, imshow, savefig
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors, NearestCentroid, LSHForest
from sklearn.decomposition import PCA, TruncatedSVD, NMF

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
feats_src2 = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Quora/data/features/NER/'
q1_vec = np.load(feats_src2 + 'q1train_NER_128len.npy')
q2_vec = np.load(feats_src2 + 'q2train_NER_128len.npy')
full_vec = np.concatenate([q1_vec, q2_vec])

src_train = 'df_train_spacylemmat_fullclean.csv'
src_test = 'df_test_spacylemmat_fullclean.csv'
df_train = pd.read_csv(src_train)
df_train.fillna('NULL', inplace = True)
#df_test = pd.read_csv(src_test)

full_questions = df_train.question1.tolist() + df_train.question2.tolist()

In [8]:
f = 128
t = AnnoyIndex(f)
start_time = time.time()

for i in tqdm(range(len(full_vec))):
    t.add_item(i, full_vec[i])
    
t.build(50)
t.save('train_doc2vec_annoymodel_50trees_angular.ann')
print('Time it took:', time.time() - start_time)

100%|██████████| 808580/808580 [00:07<00:00, 107473.41it/s]


Time it took: 395.6474838256836


In [3]:
f = 128
t = AnnoyIndex(f)
t.load('train_doc2vec_annoymodel_50trees_angular.ann')

True

In [15]:
f = 128
t2 = AnnoyIndex(f, metric = 'euclidean')
start_time = time.time()

for i in tqdm(range(len(full_vec))):
    t2.add_item(i, full_vec[i])
    
t2.build(50)
t2.save('train_doc2vec_annoymodel_50trees_euclidean.ann')
print('time it took:', time.time() - start_time)

100%|██████████| 808580/808580 [00:08<00:00, 100916.25it/s]


time it took: 181.23257184028625


In [29]:
dists_angular = []
for i in tqdm(range(len(full_vec))):
    dists_angular.append(t.get_nns_by_item(i, 10, include_distances = True)[1])


dists_euclidean = []
for i in tqdm(range(len(full_vec))):
    dists_euclidean.append(t2.get_nns_by_item(i, 10, include_distances = True)[1])

100%|██████████| 808580/808580 [02:16<00:00, 5931.70it/s]


In [31]:
def get_count_ang02(i):
    return len(i[i > 0.2])

def get_count_ang04(i):
    return len(i[i > 0.4])

def get_count_ang05(i):
    return len(i[i > 0.5])

def get_count_euc100(i):
    return len(i[i > 100])

def get_count_euc500(i):
    return len(i[i > 500])

def get_count_euc1000(i):
    return len(i[i > 1000])

def get_count_euc5000(i):
    return len(i[i > 5000])

In [32]:
dists_angular2 = np.array(dists_angular)
dists_euclidean2 = np.array(dists_euclidean)

ang_02 = np.apply_along_axis(get_count_ang02, axis = 1, arr = dists_angular2)
ang_04 = np.apply_along_axis(get_count_ang04, axis = 1, arr = dists_angular2)
ang_05 = np.apply_along_axis(get_count_ang05, axis = 1, arr = dists_angular2)

euc_100 = np.apply_along_axis(get_count_euc100, axis = 1, arr = dists_euclidean2)
euc_500 = np.apply_along_axis(get_count_euc500, axis = 1, arr = dists_euclidean2)
euc_1000 = np.apply_along_axis(get_count_euc1000, axis = 1, arr = dists_euclidean2)
euc_5000 = np.apply_along_axis(get_count_euc5000, axis = 1, arr = dists_euclidean2)

In [34]:
dists_angular2 = np.array(dists_angular)
dists_euclidean2 = np.array(dists_euclidean)

x = pd.DataFrame()
x['10nn_min_angular_q1'] = np.min(dists_angular2[:df_train.shape[0]], axis = 1)
x['10nn_mean_angular_q1'] = np.mean(dists_angular2[:df_train.shape[0]], axis = 1)
x['10nn_max_angular_q1'] = np.max(dists_angular2[:df_train.shape[0]], axis = 1)
x['10nn_min_angular_q2'] = np.min(dists_angular2[df_train.shape[0]:], axis = 1)
x['10nn_mean_angular_q2'] = np.mean(dists_angular2[df_train.shape[0]:], axis = 1)
x['10nn_max_angular_q2'] = np.max(dists_angular2[df_train.shape[0]:], axis = 1)

x['10nn_min_euclidean_q1'] = np.min(dists_euclidean2[:df_train.shape[0]], axis = 1)
x['10nn_mean_euclidean_q1'] = np.mean(dists_euclidean2[:df_train.shape[0]], axis = 1)
x['10nn_max_euclidean_q1'] = np.max(dists_euclidean2[:df_train.shape[0]], axis = 1)
x['10nn_min_euclidean_q2'] = np.min(dists_euclidean2[df_train.shape[0]:], axis = 1)
x['10nn_mean_euclidean_q2'] = np.mean(dists_euclidean2[df_train.shape[0]:], axis = 1)
x['10nn_max_euclidean_q2'] = np.max(dists_euclidean2[df_train.shape[0]:], axis = 1)

x['10nn_ang02_neighbors_q1'] = ang_02[:df_train.shape[0]]
x['10nn_ang04_neighbors_q1'] = ang_04[:df_train.shape[0]]
x['10nn_ang05_neighbors_q1'] = ang_05[:df_train.shape[0]]
x['10nn_euc100_neighbors_q1'] = euc_100[:df_train.shape[0]]
x['10nn_euc500_neighbors_q1'] = euc_500[:df_train.shape[0]]
x['10nn_euc1000_neighbors_q1'] = euc_1000[:df_train.shape[0]]
x['10nn_euc5000_neighbors_q1'] = euc_5000[:df_train.shape[0]]

x['10nn_ang02_neighbors_q2'] = ang_02[df_train.shape[0]:]
x['10nn_ang04_neighbors_q2'] = ang_04[df_train.shape[0]:]
x['10nn_ang05_neighbors_q2'] = ang_05[df_train.shape[0]:]
x['10nn_euc100_neighbors_q2'] = euc_100[df_train.shape[0]:]
x['10nn_euc500_neighbors_q2'] = euc_500[df_train.shape[0]:]
x['10nn_euc1000_neighbors_q2'] = euc_1000[df_train.shape[0]:]
x['10nn_euc5000_neighbors_q2'] = euc_5000[df_train.shape[0]:]

x.to_csv('train_10NN_distances.csv', index = False)